In [ ]:
from __future__ import division
import glob
import re
from astropy.io import fits
from astropy.io.fits import getheader
from astropy.convolution import Gaussian1DKernel as GK, convolve as cv
import numpy as np
import matplotlib.pyplot as plt
import astroscrappy
from matplotlib.patches import Rectangle
from scipy.special import i0,i1,k0,k1
from scipy.stats import norm

In [ ]:
# Create a list of all of the fits file directories
biases = glob.glob('/Users/hitom/ASTR136_Code/dark matter data/BIAS/*.fits')
arcs = glob.glob('/Users/hitom/ASTR136_Code/dark matter data/ARC/*.fits')
flats = glob.glob('/Users/hitom/ASTR136_Code/dark matter data/FLAT/*.fits')
IC4202 = glob.glob('/Users/hitom/ASTR136_Code/dark matter data/IC4202/*.fits')

In [ ]:
tmp = (fits.open(i)[0].data for i in biases)
obiases = list(tmp)
tmp = (fits.open(i)[0].data for i in arcs)
oarcs = list(tmp)
tmp = (fits.open(i)[0].data for i in flats)
oflats = list(tmp)
tmp = (fits.open(i)[0].data for i in IC4202)
oIC4202 = list(tmp)

In [ ]:
ah = getheader(arcs[0])
ah['EXPTIME']

In [ ]:
fh = getheader(flats[0])
ft = fh['EXPTIME']
ft

In [ ]:
# Master bias
Mbias = sum(obiases)/len(obiases)
# Master flat
Mflat = sum(list((i-Mbias)/ft for i in oflats))/len(oflats)
# Normalized master flat
normMflat = Mflat/np.max(Mflat)
for a in range(len(normMflat)):
    for b in range(len(normMflat[a])):
        if normMflat[a,b]==0:
            normMflat[a,b]=1
        else:
            pass
# Reduced arcs
Marc = sum(list((i-Mbias)/(normMflat) for i in oarcs))

In [ ]:
plt.figure(figsize=(14,14))
plt.imshow(normMflat,vmin=0,vmax=np.max(normMflat))
plt.colorbar();

### Here the limits of the slit are found

In [ ]:
ysums = np.zeros(len(Mflat))
for i in range(len(Mflat)):
    ysums[i] = np.sum(Mflat[i])
xsums = np.zeros(len(Mflat[0]))
for i in range(len(Mflat[0])):
    xsums[i] = np.sum(Mflat[:,i])

In [ ]:
plt.figure(figsize=(14,7))
plt.plot(ysums)
plt.xlabel('Pixel Number (Horizontal)',{'fontsize':'15'})
plt.ylabel('Sum of CCD counts',{'fontsize':'15'})
plt.savefig('FlatPixels.png')
val = 2000
for i in range(len(ysums)-1):
    if ysums[i]-ysums[i-1]>val or ysums[i]-ysums[i+1]>val:
        print(i)
;

In [ ]:
plt.figure(figsize=(14,7))
plt.plot(xsums)
val = 4000
for i in range(len(xsums)-1):
    if xsums[i]-xsums[i-1]>val or xsums[i]-xsums[i+1]>val:
        print(i)
;

In [ ]:
xl = 111
xu = 403
yl = 77
yu = 2676

In [ ]:
plt.figure(figsize=(14,14))
plt.imshow(Mflat[yl:yu,xl:xu])
plt.colorbar();

In [ ]:
Rflat = np.transpose(Mflat[yl:yu,xl:xu])
plt.figure(figsize=(14,14))
plt.imshow(Rflat,origin='lower');

In [ ]:
# Truncated Master arc
TMarc = Marc[yl:yu,xl:xu]

In [ ]:
# Finding the lines on the arc
Avals = np.zeros(len(TMarc))
for i in range(len(TMarc)):
    Avals[i] = np.sum(TMarc[i])

In [ ]:
plt.figure(figsize=(14,7))
plt.plot(Avals)
Indices = {}
UAvals = {}
n = 0
for i in range(len(Avals)):
    Threshold = 5*np.average(Avals)
    if Avals[i]>Threshold and Avals[i]>Avals[i+1] and Avals[i]>Avals[i-1]:
        Indices.update({n:i})
        print('i={0}, Avals[i]={1}'.format(i,np.format_float_scientific(Avals[i],3)))
        plt.plot(i,Avals[i],'ro')
        mean,std = norm.fit(Avals[i-10:i+10])
        lower = i-10
        upper = i+10
        for k in range(Avals[i-10:i+10].shape[0]):
            if Avals[i-10+k-1]<mean-std and Avals[i-10+k]>mean-std:
                lower = i-10+k
            if Avals[i-10+k+1]<mean-std and Avals[i-10+k]>mean-std:
                upper = i-10+k
        UAvals.update({i:upper-lower})
        n += 1
plt.xlabel('Pixel Number',{'fontsize':'15'})
plt.ylabel('Counts',{'fontsize':'15'})
#plt.savefig('ArcComp.png',bbox='tight')
;

In [ ]:
rel = {116:5875.62,1918:7032.41,2256:7245.16,2142:7173.94,\
       1756:6929.41,1426:6717.04,1366:6678.28,1244:6598.95,1142:6532.88,1101:6506.53,\
       940:6402.25,910:6389.99,835:6334.4,789:6304.79,730:6266.5,654:6217.28,570:6163.59,\
       538:6143.06}

In [ ]:
Srel = {}
SUnc = {}
i = sorted(rel)
for n in range(len(rel)):
    Srel[i[n]] = rel[i[n]]
    SUnc[i[n]] = UAvals[i[n]]

In [ ]:
Pixels = [i for i in Srel]
Values = [Srel[i] for i in Srel]
Weights = [1/UAvals[i] for i in SUnc]
Uncs = [UAvals[i] for i in SUnc]
plt.figure(figsize=(14,7))
plt.errorbar(Pixels,Values,xerr=Uncs,fmt='ro')
plt.xlabel('Pixels',{'fontsize':'15'})
plt.ylabel('Wavelength ('r'$\AA$'')',{'fontsize':'15'})
p,cov = np.polyfit(Pixels,Values,1,cov=True,w=Weights)
m = p[0]
mu = cov[0][0]
b = p[1]
bu = cov[1][1]
x = np.arange(0,max(Pixels),1)
plt.plot(m*x + b)
plt.text(100,6750,'y=({0}'r'$\pm$''{1})x+({2}'r'$\pm$''{3})'.format(np.round(m,decimals=3)\
                                                                   ,np.format_float_scientific(mu,3)\
                                                                   ,np.round(b,decimals=3)\
                                                                   ,np.round(bu,decimals=3))\
         ,{'fontsize':'12'})
plt.savefig('PtW.png',bbox='tight');

In [ ]:
print(m*940+b,m*1918+b,6402.25,7032.41)

### Now the science frames are reduced and used.

In [ ]:
SIC = list((i-Mbias)/normMflat for i in oIC4202)
TIC = [SIC[0][yl:yu,xl:xu],SIC[1][yl:yu,xl:xu]]
ICA = np.empty(TIC[0].shape)
val = 2
scale = 2
for i in range(int(len(TIC[0])/val)):
    for j in range(int(len(TIC[0][0])/val)):
        if np.sum(TIC[0][val*i:val*(i+1),val*j:val*(j+1)])>scale*np.sum(TIC[1][val*i:val*(i+1),val*j:val*(j+1)]):
            ICA[val*i:val*(i+1),val*j:val*(j+1)] = TIC[1][val*i:val*(i+1),val*j:val*(j+1)]
        elif np.sum(TIC[1][val*i:val*(i+1),val*j:val*(j+1)])>scale*np.sum(TIC[0][val*i:val*(i+1),val*j:val*(j+1)]):
            ICA[val*i:val*(i+1),val*j:val*(j+1)] = TIC[0][val*i:val*(i+1),val*j:val*(j+1)]
        else:
            ICA[val*i:val*(i+1),val*j:val*(j+1)] = TIC[1][val*i:val*(i+1),val*j:val*(j+1)]
RICA = np.transpose(ICA)

In [ ]:
plt.figure(figsize=(14,14))
plt.imshow(RICA,vmin=0,vmax=np.max(RICA)/5,origin='lower',cmap='gray');

In [ ]:
_,ICclean = astroscrappy.detect_cosmics(RICA,inmask=None,cleantype='median')

In [ ]:
fig, ax = plt.subplots(figsize=(14,14))
ax.imshow(ICclean,vmin=0,vmax=np.max(ICclean)/4,origin='lower',cmap='gray')
ax.set_autoscale_on(False)
color = (0.8,0.3,0.8)
color2 = (0.3,0.8,0.8)
r1=Rectangle((0,0),ICclean.shape[1],30,edgecolor=color,facecolor=color,alpha=0.5)
ax.add_patch(r1)
r2=Rectangle((0,250),ICclean.shape[1],ICclean.shape[0]-250,edgecolor=color,facecolor=color,alpha=0.5)
ax.add_patch(r2)
r3=Rectangle((1300,0),100,ICclean.shape[0],edgecolor=color2,facecolor=color2,alpha=0.5)
ax.add_patch(r3)
plt.xlabel('Horizontal Pixel #',{'fontsize':'15'})
plt.ylabel('Vertical Pixel #',{'fontsize':'15'})
fig.savefig('OverlayedIC.png',bbox='tight');

In [ ]:
try:
    hdu = fits.PrimaryHDU(ICclean)
    hdul = fits.HDUList([hdu])
    hdul.writeto('ReducedIC4202.fits')
except:
    pass

### Take sky from top and bottom of image

In [ ]:
bkg1 = ICclean[250:ICclean.shape[0]]
bkg2 = ICclean[0:30]
sky1 = [np.mean(i) for i in np.transpose(bkg1)]
sky2 = [np.mean(i) for i in np.transpose(bkg2)]
poly = [np.polyfit(((250+ICclean.shape[0])/2,15),(sky1[i],sky2[i]),1) for i in range(len(sky1))]
x = np.arange(ICclean.shape[0])
FICsky = [poly[i][0]*x+poly[i][1] for i in range(len(poly))]
Sky_IC = np.transpose([i for i in FICsky])

In [ ]:
plt.figure(figsize=(14,14))
plt.imshow(Sky_IC,cmap='gray',vmin=0,vmax=np.max(Sky_IC)/5);

In [ ]:
ICns = ICclean-Sky_IC
plt.figure(figsize=(14,14))
plt.imshow(ICns,cmap='gray',vmin=0,vmax=np.max(ICns)/5)
plt.xlabel('Horizontal Pixel #')
plt.ylabel('Vertical Pixel #');

In [ ]:
Noise_Reg = ICns[:,1300:1400]
noisered = [np.median(i) for i in Noise_Reg]
CICns = [ICns[i]-noisered[i] for i in range(len(noisered))]
fig, ax = plt.subplots(figsize=(14,14))
ax.imshow(CICns,cmap='gray',vmin=0,vmax=np.max(CICns)/5,origin='lower')
plt.xlabel('Horizontal Pixel #',{'fontsize':'15'})
plt.ylabel('Vertical Pixel #',{'fontsize':'15'})
plt.arrow(1385,75,25,40,color='red',head_width=20)
plt.text(1325,70,'H'r'$\alpha$',{'fontsize':'14'},color='red')
plt.arrow(1515,75,-25,40,color='blue',head_width=20)
plt.text(1515,70,'N[II]',{'fontsize':'14'},color='blue')
plt.savefig('FinCle.png',bbox='tight');

In [ ]:
Rd = 4.8
Ms = 1.8e11
Eo = Ms/(2*np.pi*(Rd**2))
G = 4.102e-6

In [ ]:
def ICrotcurve(R):
    y = R/(2*Rd)
    V = y*np.sqrt(4*np.pi*G*Eo*Rd*(i0(y)*k0(y)-i1(y)*k1(y)))
    return V

In [ ]:
x = np.arange(0.1,100.4,step=0.1)
y = [ICrotcurve(i) for i in x]
plt.figure(figsize=(14,7))
plt.plot(x,y)
plt.xlabel('Distance from center (kpc)',{'fontsize':'15'})
plt.ylabel('Rotational velocity (km/s)',{'fontsize':'15'})
plt.savefig('ExpectedVel.png',bbox_inches='tight');

In [ ]:
lower_lim = 1415
upper_lim = 1445
Zoomed = np.array(CICns)[:,lower_lim:upper_lim]
Larger_Zoomed = np.array(CICns)[:,1300:1400]
Full_Zoomed = np.array(CICns)[:,1300:1500]
fig,ax = plt.subplots(figsize=(7,7))
ax.imshow(Zoomed,cmap='gray',vmin=0,vmax=np.max(Full_Zoomed)/4,origin='lower')
plt.xlabel('Horizontal Pixel #',{'fontsize':'15'})
plt.ylabel('Vertical Pixel #',{'fontsize':'15'})
plt.savefig('CenReg.png',bbox='tight');

In [ ]:
points = np.zeros((Zoomed.shape[0],2))
points[:,0] = [np.max(i) for i in Zoomed]
for i in range(Zoomed.shape[0]):
    for k in range(Zoomed.shape[1]):
        if Zoomed[i,k]==np.max(Zoomed[i]):
            points[i,1] = k

### Note: Using 1 standard deviation as range uncertainties. Also assume noise is a gaussian distribution and use variance to find error using the median of the rows.

In [ ]:
binning = 5
# Defining arrays
means = np.zeros(int(points[:,0].shape[0]/binning))
fmeans = np.zeros(int(points[:,0].shape[0]/binning))
stds = np.zeros(int(points[:,0].shape[0]/binning))
xvals = np.zeros(int(points[:,0].shape[0]/binning))
yvals = np.zeros(int(points[:,0].shape[0]/binning))
vncs = np.zeros(int(points[:,0].shape[0]/binning))
ranges = np.zeros((int(points[:,0].shape[0]/binning),2))
variances = np.zeros(int(points[:,0].shape[0]/binning))
nmeans = np.zeros(int(points[:,0].shape[0]/binning))
uncweights = np.zeros(int(points[:,0].shape[0]/binning))
# This is the interval for which the data is in one standard deviation
rge = norm.interval(0.6745)
# This defines the box and gaussian properties of the binning box
for i in range(int(points[:,0].shape[0]/binning)):
    yrge = np.arange(points[:,0].shape[0])[binning*i:binning*(i+1)]
    xcen = np.mean(points[binning*i:binning*(i+1),1])
    ycen = i*5/2
    xrge = np.arange(Zoomed.shape[0])[int(xcen-5):int(xcen+5)]
    try:
        temp = [l for l in Zoomed[int(np.min(yrge)):int(np.max(yrge)),int(np.min(xrge)):int(np.max(xrge))]]
    except:
        continue
    vls = np.zeros((len(temp),len(temp[0])))
    for k in range(len(temp)):
        vls[k,0:len(temp[0])] = temp[k][0:len(temp[0])]
    comp = [np.mean(vls[:,l]) for l in range(len(vls[0]))]
    mean,std = norm.fit(comp)
    means[i] = mean
    stds[i] = std
    xvals[i] = xcen
    yvals[i] = ycen
    # Take the mean and median of the Zoomed for uncertainty weighing later since areas with
    # a mean near the median will not have a significantly bright area due to the size of the
    # box
    uncweights[i] = np.var(Zoomed[binning*i:binning*(i+1)])
    # Use the clear region next to the center to find noise profiles
    nmeans[i] = np.mean(Larger_Zoomed[binning*i:binning*(i+1)])
    variances[i] = np.var([k-nmeans[i] for k in Larger_Zoomed[binning*i:binning*(i+1)]])
# This calculates x ranges and uncertainty
# Since uncertainty due to noise increases as you get closer, noise uncertainty is a correlated 
# factor in the gaussian uncertainty
uncs = [np.sqrt((variances[i]+stds[i]**2))*variances[i]/uncweights[i] for i in range(len(variances))]
Uyvals = 2.5 # Due to binning

In [ ]:
low_cut = 11
high_cut = 40
xs = xvals[low_cut:high_cut]
ys = yvals[low_cut:high_cut]
ha = m*(np.mean(xs)+lower_lim)+b
uha = (np.max(xs)-np.min(xs))/(2*np.sqrt(len(xs)))
print(ha,uha)
y = [m*i+ha-(Zoomed.shape[1]/2)*m for i in xs]
yrg = np.zeros(len([i for i, val in enumerate([k>ha for k in y]) if val]))
yunc = np.zeros(yrg.shape[0])
vvals = np.zeros(yrg.shape[0])
pa = np.zeros((yrg.shape[0],2))
cit = 0
for i in range(len(y)):
    if y[i]>ha:
        yrg[cit] = y[i]
        vvals[cit] = ys[i]
        yunc[cit] = m*uncs[low_cut:high_cut][i]
        pa[cit,0] = points[low_cut:high_cut,0][i]
        pa[cit,1] = points[low_cut:high_cut,1][i]
        cit += 1
plt.figure(figsize=(14,7))
plt.errorbar(yrg,pa[:,0],xerr=yunc,fmt='ro')
plt.xlabel('Wavelength ('r'$\AA$'')',{'fontsize':'15'})
plt.ylabel('Counts',{'fontsize':'15'})
plt.savefig('Binning.png',bbox='tight');

In [ ]:
D = 100400
rpp = 0.43*np.pi/(180*3600)
kpp = D*np.tan(rpp)
maxrad = rpp*len(CICns[0])/2
Radius = [kpp*(i-np.median(yvals)) for i in yvals]

In [ ]:
muncs = [m*i for i in uncs]
plt.figure(figsize=(2,7))
plt.errorbar(m*(xvals-np.median(xvals)),Radius,xerr=muncs,fmt='ro')
plt.xlabel('Angstroms from Center',{'fontsize':'15'})
plt.ylabel('Radius (kpc)',{'fontsize':'15'})
plt.savefig('Points.png',bbox_inches='tight');

In [ ]:
l = 11
u = 40
plt.figure(figsize=(2,7))
plt.errorbar(m*(xvals[l:u]-np.mean(xvals)),Radius[l:u],xerr=muncs[l:u],fmt='ro')
plt.xlabel('Angstroms from Center',{'fontsize':'15'})
plt.ylabel('Radius (kpc)',{'fontsize':'15'});

In [ ]:
c = 2.998e5
har = 6562.8
R = kpp*2599/2
zi = 0.0238
uzi = uha/har
vsys = c*zi
uvsys = c*uzi
distances = [-kpp*(i-np.median(yvals)) for i in vvals]
Udistances = np.full(len(Radius),kpp*Uyvals)
vint = [(i-har)*c/har for i in yrg]
Uvint = [i*c/har for i in yunc]
ri = np.pi/2
vrot = [(vint[i]-vsys) for i in range(len(vint))]
Uvrot = Uvint
a = np.arange(0.1,100.4,step=0.1)
e = [ICrotcurve(i) for i in x]
plt.figure(figsize=(14,7))
plt.errorbar(distances,vrot,yerr=Uvrot,fmt='ro',label='Calculated 'r'$v_{rot}$')
plt.plot(a,e,label='Modeled 'r'$v_{rot}$')
plt.xlabel('Distance from center (kpc)',{'fontsize':'15'})
plt.ylabel('Rotational velocity (km/s)',{'fontsize':'15'})
plt.xlim(-0.5,np.max(distances)+1)
plt.legend(loc=4)
plt.savefig('RotVel.png',bbox_inches='tight');

In [ ]:
chisq = np.sum([((vrot[i]-ICrotcurve(distances[i]))**2)/((Uvrot[i]**2)*len(vrot)) for i in range(len(vrot))])
print(chisq)

In [ ]:
VelWeights = [i for i in pa[:,0]]
Mults = [VelWeights[i]*vrot[i] for i in range(len(vrot))]
FinRotVel = np.sum(Mults)/np.sum(VelWeights)
UncVels = [1/(i**2) for i in Uvrot]
FRVunc = 1/np.sqrt(np.sum([VelWeights[i]*UncVels[i] for i in range(len(UncVels))])/np.sum(VelWeights))
print(FinRotVel,FRVunc)

In [ ]:
low_cut = 11
high_cut = 45
xs = xvals[low_cut:high_cut]
ys = yvals[low_cut:high_cut]
ha = m*(np.mean(xs)+lower_lim)+b
uha = (np.max(xs)-np.min(xs))/(2*np.sqrt(len(xs)))
print(ha,uha)
y2 = [m*i+ha-(Zoomed.shape[1]/2)*m for i in xs]
yrg2 = np.zeros(len([i for i, val in enumerate([k<ha for k in y2]) if val]))
yunc2 = np.zeros(yrg2.shape[0])
vvals2 = np.zeros(yrg2.shape[0])
pa2 = np.zeros((yrg2.shape[0],2))
cit = 0
for i in range(len(y2)):
    if y2[i]<ha:
        yrg2[cit] = y2[i]
        vvals2[cit] = ys[i]
        yunc2[cit] = m*uncs[low_cut:high_cut][i]
        pa2[cit,0] = points[low_cut:high_cut,0][i]
        pa2[cit,1] = points[low_cut:high_cut,1][i]
        cit += 1
plt.figure(figsize=(14,7))
plt.errorbar(yrg2,pa2[:,0],xerr=yunc2,fmt='ro')
plt.xlabel('Wavelength ('r'$\AA$'')',{'fontsize':'15'})
plt.ylabel('Counts',{'fontsize':'15'});

In [ ]:
distances2 = [kpp*(i-np.median(yvals)) for i in vvals2]
Udistances2 = np.full(len(Radius),kpp*Uyvals)
vint2 = [(i-har)*c/har for i in yrg2]
Uvint2 = [i*c/har for i in yunc2]
ri = np.pi/2
vrot2 = [-(vint2[i]-vsys) for i in range(len(vint2))]
Uvrot2 = Uvint2
a = np.arange(0.1,100.4,step=0.1)
e = [ICrotcurve(i) for i in x]
plt.figure(figsize=(14,7))
print(len(distances2),len(vrot2),len(Uvrot2))
plt.errorbar(distances2,vrot2,yerr=Uvrot2,fmt='ro')
plt.plot(a,e)
plt.xlabel('Distance from center (kpc)',{'fontsize':'15'})
plt.ylabel('Rotational velocity (km/s)',{'fontsize':'15'})
plt.xlim(-0.1,np.max(distances)+0.5);

In [ ]:
chisq = np.sum([((vrot2[i]-ICrotcurve(distances2[i]))**2)/((Uvrot2[i]**2)*len(vrot2)) for i in range(len(vrot2))])
print(chisq)

In [ ]:
VelWeights2 = [i for i in pa[:,0]]
Mults2 = [VelWeights2[i]*vrot2[i] for i in range(len(vrot2))]
FinRotVel2 = np.sum(Mults2)/np.sum(VelWeights2)
UncVels2 = [1/(i**2) for i in Uvrot2]
FRVunc2 = 1/np.sqrt(np.sum([VelWeights2[i]*UncVels2[i] for i in range(len(UncVels2))])/np.sum(VelWeights2))
print(FinRotVel2,FRVunc2)

In [ ]:
def DC(R,genvrot,Rc):
    p = (genvrot**2)/(4*np.pi*G*(Rc**2)*(1+(R/Rc)**2))
    return p

In [ ]:
vsim = 7123.1-7038
dummy = np.arange(0.1,100.1,step=0.1)
ODMM = [DC(distances[i],vrot[i],Rd) for i in range(len(vrot))]
UODMM = [2*Uvrot[i]*DC(distances[i],vrot[i],Rd)/vrot[i] for i in range(len(vrot))]
DMM3 = [DC(i,ICrotcurve(i),Rd) for i in dummy]

In [ ]:
plt.figure(figsize=(14,7))
plt.plot(dummy,DMM3,label='Expected Dark Matter Content')
plt.errorbar(distances,ODMM,yerr=UODMM,fmt='ro',label='Calculated Dark Matter Content')
plt.xlabel('Radius(kpc)',{'fontsize':'15'})
plt.ylabel(r'$\rho_c(R)$'' ('r'$\frac{M_{\odot}}{{kpc}^3}$'')',{'fontsize':'15'})
plt.xlim(-0.5,np.max(distances)+1)
plt.legend()
plt.savefig('DMM.png',bbox_inches='tight');

In [ ]:
chisq = np.sum([((ODMM[i]-DC(distances[i],ICrotcurve(distances[i]),Rd))**2)/((UODMM[i]**2)*(len(ODMM))) for i in range(len(ODMM))])
print(chisq)